# Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this notebook, I will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions will be used:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. I will use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.
* **`doc_to_synsets:`** returns a list of synsets in document. This function first tokenizes and part of speech tags the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it finds each token corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match is used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, it finds the synset in s2 with the largest similarity value. it sums all of the largest similarity values together and normalizes this value by dividing it by the number of largest similarity values found. Missing values are ignored.


In [24]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    #"""Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    from nltk.stem import WordNetLemmatizer
#    """
#    Returns a list of synsets in document.

#    Tokenizes and tags the words in the document doc.
#    Then finds the first synset for each word/tag combination.
#    If a synset is not found for that combination it is skipped.

#    Args:
#        doc: string to be converted

#    Returns:
#        list of synsets

#    Example:
#        doc_to_synsets('Fish are nvqjp friends.')
#        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
#    """
    

    # Your Code Here
    doc_tokens = nltk.word_tokenize(doc)
    pos_tag = nltk.pos_tag(doc_tokens)
    converted_tag = [(i[0],convert_tag(i[1])) for i in pos_tag]
    synseting = [wn.synsets(token, tag) for (token, tag) in converted_tag]
    Synvalues = [val[0] for val in synseting if len(val) > 0]
    
    return Synvalues# Your Answer Here

def similarity_score(s1, s2):
#    """
#    Calculate the normalized similarity score of s1 onto s2

#    For each synset in s1, finds the synset in s2 with the largest similarity value.
#    Sum of all of the largest similarity values and normalize this value by dividing it by the
#    number of largest similarity values found.

#    Args:
#        s1, s2: list of synsets from doc_to_synsets

#    Returns:
#        normalized similarity score of s1 onto s2

#    Example:
#        synsets1 = doc_to_synsets('I like cats')
#        synsets2 = doc_to_synsets('I like dogs')
#        similarity_score(synsets1, synsets2)
#        Out: 0.73333333333333339
#    """
    
    
    # Your Code Here
    similar = []
    for i in s1:
        scores = [x for x in [i.path_similarity(j) for j in s2] if x]
        if scores: similar += [(float(max(scores)))]
    return sum(similar)/len(similar)# Your Answer Here
    

def document_path_similarity(doc1, doc2):
#    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2


### test_document_path_similarity

I Use this function to check if doc_to_synsets and similarity_score are correct.

*This function returns the similarity score as a float.*

In [25]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'I Use this function to see if my code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

test_document_path_similarity()


0.5290391156462585

### Loading Data
`paraphrases.csv` contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [26]:
# I use this dataframe for most_similar_docs and label_accuracy
import os

my_file = os.path.join("Resources", "paraphrases.csv")
paraphrases = pd.read_csv(my_file)
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

Using `document_path_similarity`, I find the pair of documents in paraphrases which has the maximum similarity score.

*This function returns a tuple `(D1, D2, similarity_score)`*

In [27]:
def most_similar_docs():
    df = paraphrases
    df['similarity'] = df.apply(lambda x: document_path_similarity(x['D1'], x['D2']), axis = 1)
    mostlarge = df.nlargest(1, 'similarity').reset_index()
    return (mostlarge.loc[0]['D1'],mostlarge.loc[0]['D2'],mostlarge.loc[0]['similarity'])

most_similar_docs()


('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9753086419753086)

### label_accuracy

I will provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. the classifier rule is that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). I will report accuracy of the classifier using scikit-learn's accuracy_score.

*This function returns a float.*

In [28]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    # Your Code Here
    df = paraphrases
    df['similarity'] = df.apply(lambda x: document_path_similarity(x['D1'], x['D2']), axis = 1)
    df['NewQuality'] = df.apply(lambda x: 1 if x['similarity']>0.75 else 0, axis = 1)
    return accuracy_score(df['Quality'], df['NewQuality'])

label_accuracy()


0.8

## Part 2 - Topic Modelling

For the second part of this notebook, I will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. I will first use the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. I will then extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [29]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
newsgroups = os.path.join("Resources", "newsgroups")
with open(newsgroups, 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [30]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word=id_map, num_topics=10, passes=25, random_state=34)

### lda_topics

Using `ldamodel`, We find a list of the 10 topics and the most significant 10 words in each topic. This is structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function returns a list of tuples.*

In [31]:
def lda_topics():
    topics_words = ldamodel.print_topics(num_topics=10, num_words=10)
    return topics_words

lda_topics()

[(0,
  '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'),
 (1,
  '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'),
 (2,
  '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'),
 (3,
  '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'),
 (4,
  '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'),
 (5,
  '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'),
 (6,
  '0.0

### topic_distribution

For the new document `new_doc`, I will find the topic distribution. I will use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function returns a list of tuples, where each tuple is `(#topic, probability)`*

In [32]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [33]:
def topic_distribution():
    new_doc_transformed = vect.transform(new_doc)
    corpus = gensim.matutils.Sparse2Corpus(new_doc_transformed, documents_columns=False)
    doc_topics = ldamodel.get_document_topics(corpus)
    topic_dist = []
    for val in list(doc_topics):
        for v in val:
            topic_dist += [v]
    return topic_dist

topic_distribution()

[(0, 0.020001832),
 (1, 0.020002048),
 (2, 0.020000001),
 (3, 0.4963116),
 (4, 0.020002764),
 (5, 0.020002857),
 (6, 0.020001696),
 (7, 0.020001369),
 (8, 0.020001847),
 (9, 0.34367394)]

### topic_names

We assign topic names to the topics found from below list. If none of these names best matches the topics found, I will create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [34]:
topic_names = ["Education", "Automobiles", "Computers & IT", "Religion", "Automobiles", "Sports", "Health", "Religion", "Computers & IT", "Science"]
topic_names

['Education',
 'Automobiles',
 'Computers & IT',
 'Religion',
 'Automobiles',
 'Sports',
 'Health',
 'Religion',
 'Computers & IT',
 'Science']